<a href="https://colab.research.google.com/github/smannan/LIFXBulbAnalysis/blob/main/SmartBulbDataExtractFromInflux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Query LIFX bulb data from an Influx DB client

1.   LIFX API spec: https://api.developer.lifx.com/docs/list-lights
2.   Visualize bulb brightness and power over time
3.   Will need Influx acces token, org, and bucket
4.   For info on Influx python client: https://docs.influxdata.com/influxdb/cloud/api-guide/client-libraries/python/

In [1]:
pip install influxdb-client

In [2]:
POWER_ENUM = { 'on': 1, 'off': 0 }

In [3]:
import pandas as pd

Create Influxe DB client

In [4]:
from datetime import datetime

from influxdb_client import InfluxDBClient, Point, WritePrecision

# You can generate a Token from the "Tokens Tab" in the UI
# https://us-central1-1.gcp.cloud2.influxdata.com/orgs/a33284f1a3112d0e/data-explorer
token = "BMNyseUoGWt6hU5tHxgYh6jyEkAYXAX514CIUtWhHmdu9nshYBAk75Oc1YBJOp84WyFL_4HIxdgkSkTpXvl_NA=="
org = "praveenkumar23.anguru@gmail.com"
bucket = "SmartBulbData"

client = InfluxDBClient(url="https://us-central1-1.gcp.cloud2.influxdata.com", token=token, verify_ssl=False)

In [5]:
### Print number of tables, records per table, and example record for each
### table returned by a query
def print_table_stats(tables):
  # one table per light bulb
  print ('Num tables {0}\n'.format(len(tables)))
  for table in tables:
    print ('Num records in table {0}'.format(len(table.records)))
    if len(table.records) > 0:
      record = table.records[0]
      print ('Sample record {0}, {1}\n'.format(record.get_field(), record.get_value()))

In [84]:
### Query a week's worth of data from an Influx db bucket for a specific field
def query_field(field):
  query = f'from(bucket: \"{bucket}\") |> range(start: -14d) |> filter(fn:(r) => r._field == \"{field}\" )'
  print (query)
  tables = client.query_api().query(query, org=org)
  print_table_stats(tables)
  return tables

Get a week's worth of power data from the database

In [87]:
tables = query_field('brightness')

from(bucket: "SmartBulbData") |> range(start: -14d) )


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


ReadTimeoutError: ignored

In [74]:
# table_id = 0 or 1 depending on which bulb
### Put the each record's value and time information into a pandas dataframe
def get_field_values_from_flux(field):
  tables = query_field(field)
  field_values = []

  for table in tables:
    for record in table.records:
      value = record.get_value()

      field_values.append({
          'timestamp': record.get_time(),
          'value': value,
          'id': record.values['id']
      })

  print ('Extracted {0} values'.format(len(field_values)))
  print ('Sample {0}\n'.format(field_values[0]))

  field_values = pd.DataFrame(field_values)
  print ('Earliest: {0}'.format(field_values['timestamp'].min()))
  print ('Latest: {0}'.format(field_values['timestamp'].max()))

  return field_values

Put a week's worth of power data into a pandas dataframe

In [77]:
df_formatted = get_field_values_from_flux('brightness')

from(bucket: "SmartBulbData") |> range(start: -14d) |> filter(fn:(r) => r._field == "brightness" )


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Num tables 2

Num records in table 2809
Sample record brightness, 0.607

Num records in table 2399
Sample record brightness, 0.3906

Extracted 5208 values
Sample {'timestamp': datetime.datetime(2021, 8, 27, 2, 34, tzinfo=tzlocal()), 'value': 0.607, 'id': 'd073d55df12f'}

Earliest: 2021-08-27 02:34:00+00:00
Latest: 2021-09-09 05:54:03+00:00


In [79]:
df_formatted.head()
df_formatted.to_csv('smart_bulb_data_from_influx_aug27_sept9.csv')